In [2]:
import pandas as pd
import numpy as np
from matplotlib.image import imread
from os import listdir
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
folder = './data/images/'
food_types = listdir(folder)

In [3]:
T = pd.read_csv('calories/en.openfoodfacts.org.products.tsv', sep='\t')

def getAverage(T, t):
    t = t.replace('_', ' ').lower()
    A = np.array([(i, similar(t,product.lower()), product) 
                  for i, product in enumerate(T['product_name'].astype(str))])
    test = np.flip(A[A[:,1].argsort()])
    indices = [int(t[2]) for t in test[np.transpose(test)[1].astype(float) > 0.9]]
    return T[T.index.isin(indices)].mean()

averages = [getAverage(T, food) for food in food_types]
df = pd.DataFrame(averages)
df.insert(0, 'food_type', food_types)
df = df[df.columns[df.apply(lambda x: sum(x.isna()) / len(x)) < 0.3]]
df = df.drop(['ingredients_from_palm_oil_n', 
              'ingredients_that_may_be_from_palm_oil_n'], axis=1)
df = df.dropna()
df['calories_100g'] = df['energy_100g'] / 4.184
df.drop('energy_100g',1, inplace=True)
#df.to_csv('food_nutrition.csv', index=False)

KeyboardInterrupt: 